In [2]:
import pandas as pd
import numpy as np
import warnings

In [3]:
warnings.filterwarnings('ignore')

In [57]:
# From TPD
tracks = pd.read_csv('track.csv',sep=';', error_bad_lines=False)
lastfm_rating = pd.read_csv('lastfm_rating.csv', sep=';')
billboard_rating = pd.read_csv('billboard_rating.csv',sep=';')
spotify_rating = pd.read_csv('spotify_rating.csv',sep=';')
artist_track = pd.read_csv('artist_track.csv',sep=';')
artists = pd.read_csv('artist.csv',sep=';')

# From Hot100
hot100 = pd.read_csv('Hot100.csv')
hot100_complete = pd.read_csv('hot100_complete_withcount.csv')
hot5870_sum = pd.read_csv('hot5870_sum.csv')
hot5870_withcount = pd.read_csv('hot5870_withcount.csv')
hot5870_playcount = pd.read_csv('hot5870_playcount.csv'); 

# From fma_metadata
echonest = pd.read_csv('echonest.csv',error_bad_lines=False)
genres = pd.read_csv('genres.csv',error_bad_lines=False)


In [4]:
tracks_tmp = tracks[['id', 'title', 'release_date']]
tracks_tmp = tracks_tmp[tracks_tmp['release_date']<'1970-01-01'] 
tracks_tmp.columns = ['id', 'title', 'release_date']


In [5]:
lastfm_rating_tmp = lastfm_rating[['id', 'date', 'position']]
lastfm_rating_tmp.columns = ['id', 'date_lastfm', 'position_lastfm']

In [6]:
billboard_rating_tmp = billboard_rating[['id', 'date', 'position']]
billboard_rating_tmp.columns = ['id', 'date_billboard', 'position_billboard']

In [7]:
spotify_rating_tmp = spotify_rating[['id', 'date', 'position']]
spotify_rating_tmp.columns = ['id', 'date_spotify', 'position_spotify']

In [8]:
Final = pd.merge(pd.merge(pd.merge(tracks_tmp,lastfm_rating_tmp,on='id'),billboard_rating_tmp,on='id'),spotify_rating_tmp,on='id').head()

In [9]:
Final_lastfm = pd.merge(tracks_tmp,lastfm_rating_tmp,on='id')
Final_lastfm.shape

(1267, 5)

In [10]:
Final_billboard = pd.merge(tracks_tmp,billboard_rating_tmp,on='id')
Final_billboard.shape

(1267, 5)

In [11]:
Final_spotify = pd.merge(tracks_tmp,spotify_rating_tmp,on='id')
Final_spotify.shape

(842, 5)

In [12]:
Final_spotify_lastfm = pd.merge(lastfm_rating_tmp,spotify_rating_tmp,on='id')
Final_spotify_lastfm.shape

(6500, 5)

In [13]:
AA = spotify_rating[spotify_rating['date']<'1970-01-01']
AA.shape

(0, 5)

In [14]:
BB = billboard_rating[billboard_rating['date']<'1970-01-01']
BB.shape

(0, 4)

### Investigating Hot100

In [15]:
print("hot100complete : ",hot100_complete.columns)
print("hot5870 : ",hot5870_sum .columns)
print("hot5870_withcount : ",hot5870_withcount.columns)

hot100complete :  Index(['song', 'artist', 'sum', 'count'], dtype='object')
hot5870 :  Index(['song', 'artist', 'position'], dtype='object')
hot5870_withcount :  Index(['song', 'artist', 'sum', 'count'], dtype='object')


In [16]:
print("hot100complete shape : ",hot100_complete.shape)
print("tracks shape : ",tracks.shape)

hot100complete shape :  (27142, 4)
tracks shape :  (23385, 5)


In [17]:
# rename columns
tracks.columns = ['track_id', 'song', 'release_date', 'lastfm_playcount', 'lastfm_listeners']
artists.columns = ['artist_id', 'artist', 'image_url']

In [18]:
# inner join to get artist_id for each track
tmp_tracks = pd.merge(tracks[['track_id','song','release_date','lastfm_playcount', 'lastfm_listeners']],artist_track,how='inner',on=['track_id'])

In [19]:
tmp_tracks.shape

(23385, 6)

In [20]:
# inner join to get artistname of each track
new_tracks = pd.merge(tmp_tracks[['track_id','song','artist_id','release_date','lastfm_playcount', 'lastfm_listeners']],artists[['artist_id','artist']],how='inner',on=['artist_id'])

In [21]:
new_tracks.shape

(23385, 7)

In [22]:
new_tracks.head(10)

,track_id,song,artist_id,release_date,lastfm_playcount,lastfm_listeners,artist
0,1,Uptown Funk!,1,2014-11-10 00:00:00,0,0,Mark Ronson Featuring Bruno Mars
1,2,Blank Space,2,2014-10-27 00:00:00,1822265,92902,Taylor Swift
2,9,Shake It Off,2,2014-08-18 00:00:00,2479013,193297,Taylor Swift
3,72,Style,2,2014-10-27 00:00:00,1110899,63974,Taylor Swift
4,113,Bad Blood,2,2014-10-27 00:00:00,650377,54756,Taylor Swift
5,187,Welcome To New York,2,2014-10-27 00:00:00,660411,57373,Taylor Swift
6,189,Wildest Dreams,2,2014-10-27 00:00:00,657658,53726,Taylor Swift
7,194,Out Of The Woods,2,2002-02-02 00:00:00,1070115,60681,Taylor Swift
8,537,Red,2,2012-10-22 00:00:00,0,0,Taylor Swift
9,581,Sweeter Than Fiction,2,2013-10-21 00:00:00,0,0,Taylor Swift


In [23]:
# inner join hot100 with tracks to get information of songs which came up in tracks
hot100_Tracks = pd.merge(new_tracks[['song','artist','release_date','lastfm_playcount', 'lastfm_listeners']],hot100_complete,how='inner',on=['song','artist'])

In [24]:
hot100_Tracks.shape

(4856, 7)

In [25]:
hot100_Tracks.head()

,song,artist,release_date,lastfm_playcount,lastfm_listeners,sum,count
0,Uptown Funk!,Mark Ronson Featuring Bruno Mars,2014-11-10 00:00:00,0,0,650,54
1,Blank Space,Taylor Swift,2014-10-27 00:00:00,1822265,92902,572,36
2,Shake It Off,Taylor Swift,2014-08-18 00:00:00,2479013,193297,948,50
3,Style,Taylor Swift,2014-10-27 00:00:00,1110899,63974,982,32
4,Bad Blood,Taylor Swift,2014-10-27 00:00:00,650377,54756,158,2


In [68]:
# Extracting artist_hotttnesss
echonest = pd.read_csv('echonest.csv',error_bad_lines=False)
echonest_tmp = echonest
echonest_tmp.columns = echonest_tmp.iloc[1]
echonest_tmp = echonest_tmp.reindex(echonest_tmp.index.drop(0))
echonest_tmp = echonest_tmp.reindex(echonest_tmp.index.drop(1))
echonest_tmp = echonest_tmp.reindex(echonest_tmp.index.drop(2))
#echonest_tmp = echonest_tmp.drop_duplicates('artist_name')
echonest_artist_hotttnesss = echonest_tmp[['artist_name','artist_hotttnesss']]

In [27]:
print(echonest_artist_hotttnesss.shape)
print(hot100_Tracks.shape)

(13129, 2)
(4856, 7)


In [28]:
echonest_artist_hotttnesss.columns = ['artist','artist_hotttnesss']
test = hot100_Tracks
test2 = pd.merge(test,echonest_artist_hotttnesss,how='inner',on=['artist'])
test2.shape

(16, 8)

### Scoring the songs

In [29]:
hot100 = pd.read_csv('Hot100.csv')

In [50]:
hot100 = hot100.assign(weighted_position = ( hot100['position'] * (101 -hot100['position'])) )

In [52]:
hot100['date'] = pd.to_datetime(hot100['date'], errors = 'coerce') 

In [55]:
hot58_70 = hot100[(hot100['date'] > '1958-01-01') & (hot100['date'] < '1970-12-31')]

In [35]:
dd = hot100.groupby(['song','artist'])['weighted_position'].agg(['sum','count']).reset_index()

In [56]:
hot58_70.head()

,date,position,song,artist,weighted_position
0,1958-08-09,1,Poor Little Fool,Ricky Nelson,100
1,1958-08-09,2,Patricia,Perez Prado And His Orchestra,198
2,1958-08-09,3,Splish Splash,Bobby Darin,294
3,1958-08-09,4,Hard Headed Woman,Elvis Presley With The Jordanaires,388
4,1958-08-09,5,When,Kalin Twins,480


In [42]:
dd = dd.assign(score = (dd['sum'] + dd['count']))

In [48]:
dd.sort_values(['sum','count'],ascending = [False,False],inplace = True)

In [49]:
dd.head()

,song,artist,sum,count,score
18812,Sail,AWOLNATION,152556,79,152635
11190,I'm Yours,Jason Mraz,129804,76,129880
17926,Radioactive,Imagine Dragons,126670,87,126757
2272,Before He Cheats,Carrie Underwood,118936,64,119000
25164,What Hurts The Most,Rascal Flatts,112100,51,112151


### Preparing genres of tracks

In [58]:
print("tracks col : ",tracks.columns)
print("genres col : ",genres.columns)

tracks col :  Index(['id', 'title', 'release_date', 'lastfm_playcount', 'lastfm_listeners'], dtype='object')
genres col :  Index(['genre_id', '#tracks', 'parent', 'title', 'top_level'], dtype='object')


In [67]:
genres.head(10)

,track_id,genres
0,8693,Avant-Garde
1,5271,International
2,1752,Blues
3,4126,Jazz
4,4106,Classical
5,914,Novelty
6,217,Comedy
7,868,Old-Time / Historic
8,1987,Country
9,13845,Pop


In [59]:
genres.columns = ['genre_id', 'track_id', 'parent', 'genres', 'top_level']
genres.drop(['genre_id','parent','top_level'], axis = 1, inplace = True)


In [66]:
df_track_tmp = tracks
print("df_tmp_track shape : " ,df_track_tmp.shape)
print("genres : " ,genres.shape)

df_tmp_track shape :  (23385, 2)
genres :  (163, 2)


In [52]:
#df_track_tmp['release_date'] = pd.to_datetime(df_track_tmp['release_date'], errors='coerce')
#df_track_tmp = df_track_tmp[(df_track_tmp['release_date'] > '1958-01-01') &(df_track_tmp['release_date'] < '1970-12-31')]

In [55]:
#df_track_tmp.shape

(4, 5)

In [61]:
df_track_tmp.drop(['release_date', 'lastfm_playcount', 'lastfm_listeners'],axis = 1, inplace = True)
df_track_tmp.columns = ['track_id','title']

In [62]:
track_genres = pd.merge(genres,df_track_tmp,how='inner',on=['track_id'])


In [63]:
track_genres.drop(['track_id'], axis = 1, inplace = True)

In [65]:
track_genres.shape

(158, 2)

In [72]:
if 'genres' in echonest_tmp.columns:
    print("yes")
else:
    print("No")

No


In [73]:
if 'genare' in echonest_tmp.columns:
    print("yes")
else:
    print("No")

No
